In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [3]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [4]:
SYSTEM_PROMPT = """
I want you to act as software tester.
Your task is to read this information about a use case.
Then you predict all scenarios that can happen in this use case.

Rules of generating scenarios you should follow:
- Scenarios involve executing all steps within a specific flow with varied data or actions.
- Scenarios should be independent and require a complete flow to proceed.
- A scenario encompasses a whole function, not just verifying individual steps.
- Each flow must have at least one test scenario to cover the flow.
- Ensure that the test scenarios cover all the necessary aspects of the use case flow.

I only need scenarios's name for the output, I do not need the steps to go with it.
"""

In [5]:
usecase_path = r"C:\Users\congc\Desktop\GPT\Dissertation-GPT\dataset\SpecificationData\Shopping System"
save_path = r"C:\Users\congc\Desktop\GPT\Dissertation-GPT\evaluate\prove\Shopping System\ts"

In [6]:
usecase_name_list = []
project_name = os.path.basename(usecase_path)

In [7]:
for filename in os.listdir(usecase_path):
    usecase_name_list.append (filename.split(".txt")[0])
print(usecase_name_list)

['add to cart', 'checkout', 'product information display', 'purchase', 'search']


In [8]:
for usecase_name in usecase_name_list:
    print(usecase_name)
    usecase_directlink = os.path.join(usecase_path,usecase_name+".txt")
    prompt_all = read_file_content(usecase_directlink)
    promptSYSTEM_PROMPT = [
        { "role": "system", "content": SYSTEM_PROMPT},
        { "role": "user", "content": prompt_all}
    ]
    gpt_response = ask(promptSYSTEM_PROMPT, client, model)    
    print("gpt_response \n" +gpt_response)
    write_string_to_file(os.path.join(save_path,f"{usecase_name}.txt"), gpt_response)

add to cart
gpt_response 
1. Successful Item Addition with Default Quantity
2. Successful Item Addition with Increased Quantity
3. Successful Item Addition with Option Selected
4. Attempt to Add Item Without Selecting Option
5. Attempt to Add More Items Than Available in Stock
6. Attempt to Decrease Quantity Below One
7. Attempt to Add Out of Stock Item
8. Attempt to Add Item with Out of Stock Option
9. Successful Item Addition with Maximum Quantity Based on Stock
10. Successful Item Addition and Cart Icon Update
String has been written to C:\Users\congc\Desktop\GPT\Dissertation-GPT\evaluate\prove\Shopping System\ts\add to cart.txt
checkout
gpt_response 
1. Successful Checkout with Default Options
2. Checkout with Changed Shipping Address
3. Checkout with New Shipping Address
4. Checkout with Application Voucher Applied
5. Checkout with Shop Voucher Applied
6. Checkout with Changed Payment Method to Shopee Pay
7. Checkout with Changed Payment Method to Google Pay
8. Checkout with Chang